<a href="https://colab.research.google.com/github/THARANI0327/first/blob/main/cleaned_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel('/content/CC_Data_Assessment.xls')

# creating a empty row at top
df = pd.concat([pd.DataFrame([{}], columns=df.columns), df], ignore_index=True)

# removing the unwanted rows
count = 0
lister = []
for i in df['Invoice No']:
  if i=='Member : ':
    lister.append(count-1)
    count = count +1
  else:
    count = count +1

df = df.iloc[[i for i in range(len(df)) if i not in lister]].reset_index(drop=True)

# removing unwanted columns
df = df[:-3]
new_col =  ['Unnamed: 9','Unnamed: 10', 'Unnamed: 11',  'Unnamed: 12'] + [i for i in df.columns if i not in ['Unnamed: 9''Unnamed: 10', 'Unnamed: 11',  'Unnamed: 12']]
df =  df.reindex(columns=new_col)
df = df.iloc[:, :12]
df = df.rename(columns = {'Unnamed: 9': 'Member Code',
                          'Unnamed: 10': 'Member Name',
                    'Unnamed: 11' : 'Invoice Number',
                    'Unnamed: 12': 'Invoice Date',
                    'Invoice No': 'Inventory Code',
                    'Invoice Date': 'Product Name',
                    'Unnamed: 2': 'Sold Quantity',
                    'Unnamed: 3':'Sell Rate',
                    'Unnamed: 4':'Discount Amount',
                    'Unnamed: 5':'Net Amount',
                    'Unnamed: 6':'Department',
                    'Unnamed: 7':'Category'})

# finding the rows with invoice number and invoice date
def writer(value):
  count = 0
  lister = []
  for i in value:
    if i == 'InventoryCode':
      lister.append(count)
      count = count +1
    else:
      count = count +1
  return lister

# filling the invoice number and invoice date
rows = writer(df['Net Amount'])
for i in rows:
  df.iloc[i, 2] = df.iloc[i, 4]
  df.iloc[i, 3] = df.iloc[i, 5]

df['Invoice Number'].ffill(inplace=True)
df['Invoice Date'].ffill(inplace=True)

# filling the member code and member name
count = 0
lister = []
for i in df['Inventory Code']:
  if i=='Member : ':
    lister.append(count)
    count = count +1
  else:
    count = count +1

mem_code = df['Product Name'][lister].str.split(' - ').str[0].tolist()
mem_name = df['Product Name'][lister].str.split(' - ').str[1].tolist()
df.loc[lister, 'Member Code'] = mem_code
df.loc[lister, 'Member Name'] = mem_name
cash_rows = df[df['Member Code'] == 'CASH'].index
df.loc[cash_rows, 'Member Name'] = 'CASH'
df.loc[cash_rows, 'Member Code'] = np.nan
df['Member Code'].ffill(inplace=True)
df['Member Name'].ffill(inplace=True)

# removing the unwanted rows
df = df.iloc[[i for i in range(len(df)) if i not in lister]].reset_index(drop=True)

# creating sales amount column
df['Sales Amount'] = df['Sell Rate']*df['Sold Quantity']

df = df[['Invoice Date', 'Invoice Number', 'Member Code', 'Member Name', 'Inventory Code', 'Product Name', 'Category', 'Department', 'Sold Quantity', 'Sell Rate', 'Sales Amount', 'Discount Amount', 'Net Amount']]
df.head()


df.to_csv("Cleaned_Dataset1.csv", index=False)

/tmp/ipython-input-325483307.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Invoice Number'].ffill(inplace=True)
/tmp/ipython-input-325483307.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.